In [1]:
import re
import numpy as np
import pandas as pd
import pickle
from eunjeon import Mecab
from gensim.models import Word2Vec as w2v
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
#21대 국회 법안 데이터 전처리

df = pd.read_excel('21th_history.xlsx')

In [3]:
#가결/부결 코딩

for i in range(len(df)):
    if df.loc[i, '처리구분'] == 1 or df.loc[i, '처리구분'] == 2 or df.loc[i, '처리구분'] == 3 or df.loc[i, '처리구분'] == 4:
        df.loc[i, '처리구분'] = 1
    elif df.loc[i, '처리구분'] == 5 or df.loc[i, '처리구분'] == 9:
        df.loc[i, '처리구분'] = 1
    elif df.loc[i, '처리구분'] == 6 or df.loc[i, '처리구분'] == 7 or df.loc[i, '처리구분'] == 8:
        df.loc[i, '처리구분'] = 0
    elif df.loc[i, '처리구분'] == 10:
        df.loc[i, '처리구분'] = 0

In [4]:
#소관위 원-핫코딩
committee_set = list(set(df['소관위']))
committee = []
for com in committee_set:
    coms = com.split(", ")
    for c in coms:
        committee.append(c)
        
committee = list(set(committee))

for i in range(len(df)):
    for com in committee:
        if df['소관위'][i].__contains__(com):
            df.loc[i, com] = 1
        else:
            df.loc[i, com] = 0

In [6]:
for i in range(len(df)):
    if df.loc[i, '소속정당 구분'] == '더불어민주당' or df.loc[i, '소속정당 구분'] == '열린민주당':
        df.loc[i, 'party'] = 0
    elif df.loc[i, '소속정당 구분'] == '국민의힘' or df.loc[i, '소속정당 구분'] == '미래통합당' :
        df.loc[i, 'party'] = 1
    elif df.loc[i, '소속정당 구분'] == '국민의당' or df.loc[i, '소속정당 구분'] == '기본소득당' or df.loc[i, '소속정당 구분'] == '시대전환' or df.loc[i, '소속정당 구분'] == '정의당':
        df.loc[i, 'party'] = 2
    else:
        df.loc[i, 'party'] = 3

In [7]:
for i in range(len(df)):
    if df.loc[i, '소속정당 구분'] == '더불어민주당' or df.loc[i, '소속정당 구분'] == '열린민주당' :
        df.loc[i, 'num_seats'] = 185
    elif df.loc[i, '소속정당 구분'] == '국민의힘' :
        df.loc[i, 'num_seats'] = 103
    elif df.loc[i, '소속정당 구분'] == '미래통합당' :
        df.loc[i, 'num_seats'] = 84
    elif df.loc[i, '소속정당 구분'] == '정의당' or df.loc[i, '소속정당 구분'] == '기본소득당' or df.loc[i, '소속정당 구분'] == '시대전환':
        df.loc[i, 'num_seats'] = 185
    elif df.loc[i, '소속정당 구분'] == '국민의당':
        df.loc[i, 'num_seats'] = 3
    elif df.loc[i, '소속정당 구분'] == '무소속':
        df.loc[i, 'num_seats'] = 9

In [8]:
#법안별 주요내용이 26개 카테고리 주제와 얼마나 유사한지 유사도 계산

model = w2v.load('word2vec.model') #모델 불러오기

tagger = Mecab()

summary_re=[]
for text in df['제안내용']:
    try:
        summary_re.append(re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text))
    except:
        summary_re.append('NaN')
    
cnt_words = []
summary_re3 = []

for text in summary_re:
    pos_text = tagger.pos(text)
    word_list = []
    for word in pos_text:
        word_list.append(word[0])
    cnt_words.append(len(word_list)+1)
    summary_re3.append(' '.join(word_list))
    
df['제안내용_re']=summary_re3

In [9]:
words = list()
for word in model.wv.vocab:
    words.append(word)

#주제 카테고리 dic
category = {'국회': ['의회', '국회'],
            '정당/선거': ['정당', '선거'] ,
            '안보': ['군사', '안보', '북한'], 
            '사법': ['사법', '법원', '검찰', '소송'],
            '행정' : ['행정', '지자체', '자치', '경찰', '공무원'],
            '재정' : ['재정', '예산', '회계'], 
            '중소기업' : ['중소기업', '창업', '벤처', '스타트업'],
            '에너지' : ['에너지', '수소', '전기', '가스'], 
            '부동산' : ['주택', '부동산', '주거', '임대차'], 
            '금융' : ['금융', '투자', '자본', '보험업'], 
            '자동차' : ['자동차', '승합자동차', '이륜자동차', '승용차'],
            '건설/기계/조선' : ['기계', '건설업', '건설사', '건축', '조선업', '해운업', '발주'],
            '유통/무역' : ['유통', '물류', '무역', '쇼핑몰', '마트', '백화점'], 
            'IT' : ['IT', '통신', '게임', '데이터', '인공지능', '블록체인', '클라우드'], 
            '농축산' : ['농업', '축산', '수산'],
            '복지':['복지', '연금', '빈곤', '수당'],
            '의료/보건' : ['의료', '보건', '병원', '질병', '의약품'] , 
            '도시/교통':['도시', '교통', '운전', '도로'],
            '교육' : ['교육', '학교', '대학', '유치원', '입시'], 
            '환경' : ['친환경', '오염', '저탄소', '온실가스', '기후', '수자원'], 
            '노동' : ['노동', '임금', '노동조합', '퇴직', '채용', '근로'], 
            '치안/안전' : ['치안', '범죄', '안전', '사고', '소방', '형사'],  
            '가족' : ['가족', '아동', '청소년'],
            '여성' : ['여성', '출산', '육아', '성범죄', '성희롱'], 
            '예체능' : ['예술', '영화', '음악', '전시','공연', '문화재', '체육', '방송', '언론', '스포츠', '콘텐츠']
            }


for key in category.keys():
    keyword_similar = np.zeros(shape=(len(words), ))
    
    for keyword in category[key]:
        keyword_similar_2 = []
        for word in words:
            sm = model.wv.similarity(keyword, word)
            if sm >= 0.8:
                keyword_similar_2.append(sm)
            else:
                keyword_similar_2.append(0)

        keyword_similar_2 = np.asarray(keyword_similar_2)
        keyword_similar += keyword_similar_2
    
    keyword_similar = list(keyword_similar)
    keyword_similar_df = pd.DataFrame({'words': words, 'similar' : keyword_similar})
    keyword_similar_df = keyword_similar_df.sort_values(by=['words'], axis=0)
    keyword_similar_df = keyword_similar_df.set_index('words')
    keyword_similar_df = keyword_similar_df.T

    vectorizer = CountVectorizer(vocabulary = keyword_similar_df.columns, binary=False)
    X = vectorizer.fit_transform(summary_re3)
    words_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
    words_df = words_df.T

    tdm = np.dot(keyword_similar_df, words_df)
    tdm = tdm.T
    tdm = np.ravel(tdm) #2차원 배열을 1차원으로 평평하게 만듬
    df[key] = list(tdm / cnt_words)

In [10]:
#공동발의데이터 전처리
spon = pd.read_csv('21th_sponsorship_0104.csv')
spon = spon.loc[ spon['의안번호'].isin(df['법안코드']) ].reset_index().drop(['index'],axis=1)
spon = spon[spon['구분']!='대표'].fillna(0)
spon = spon[spon['당선횟수']!=0]

In [11]:
#공동발의 의원들의 평균 선수
avg_elected = spon.pivot_table(index=['의안번호'], values=['당선횟수'], aggfunc=np.mean)
avg_elected = avg_elected.reset_index()

In [12]:
#정당다양성 변수: 공동발의 의원이 모두 같은 당 소속이면 0, 아니면 1

party_set = []

bill_no = list(set(spon['의안번호']))

for no in bill_no:
    bill = spon[spon['의안번호']==no]
    parties = set(bill['정당'])
    if len(parties) >= 2:
        party_set.append(1)
    else:
        party_set.append(0)
        
diversity = pd.DataFrame({'의안번호': bill_no, '정당다양성':party_set})

In [13]:
extra_var = pd.merge(avg_elected, diversity, how='left', on='의안번호')
extra_var = extra_var.rename(columns={"당선횟수": "공동발의평균선수"})
extra_var = extra_var.rename(columns={"의안번호": "법안코드"})

df = pd.merge(df, extra_var, how='left', on='법안코드')

In [14]:
#상임위 상정 여부 변수

for i in range(len(df)):
    if type(df.loc[i, '위원회 상정일']) == pd._libs.tslibs.timestamps.Timestamp:
        df.loc[i, '상임위 상정 여부'] = 1
    else:
        df.loc[i, '상임위 상정 여부'] = 0

In [34]:
#소관위에서 대표발의자와 다른 정당 의원이 발언한 횟수 변수화하기

hearings = pd.read_excel('hearings_count.xlsx') #회의록 데이터
hearings = hearings.rename(columns={"법안번호": "법안코드"})
hearings = hearings.loc[ hearings['법안코드'].isin(df['법안코드']) ].reset_index().drop(['index'],axis=1)
hearings_df = pd.merge(hearings, df, how='left', on='법안코드')

In [35]:
from datetime import datetime

party_21th = pd.read_csv('party_21th.csv')
party_21th = party_21th[['emp_nm', 'poly_nm']]
party_21th = party_21th.rename(columns={'emp_nm':'화자'})
df_merged = pd.merge(hearings_df, party_21th, how='left', on='화자')
df_merged = df_merged.rename(columns={'정당':'제안자 정당'}).reset_index()
df_merged = df_merged.rename(columns={'poly_nm':'화자 정당'})

In [36]:
df_merged['상임위원장 정당'] = '더불어민주당'

for i in range(len(df_merged)):
    if df_merged.loc[i, '제안자'] == '위원장': #위원장이 발의한 경우 상임위원장 정당을 제안자 정당으로 본다
        df_merged.loc[i, '제안자 정당'] = df_merged.loc[i, '상임위원장 정당']
        
for i in range(len(df_merged)):
    if df_merged.loc[i, '제안자'] == '정부':   #정부가 발의한 경우 더불어민주당(20대 국회 하반기: 문재인 정부)을 제안자 정당으로 본다
        df_merged.loc[i, '제안자 정당'] = '더불어민주당'

In [37]:
bill_no = list(set(df_merged['법안코드']))

cnt_list = []

for no in bill_no:
    bill_df = df_merged[df_merged['법안코드']==no].reset_index()
    cnt = 0
    for i in range(len(bill_df)):
        if bill_df['제안자 정당'][0] != bill_df.loc[i, '화자 정당'] and type(bill_df.loc[i, '화자 정당']) == str:
            cnt+=bill_df.loc[i, '횟수']
    cnt_list.append(cnt)

bill_cnt = pd.DataFrame({'법안코드':bill_no, '반대당발언횟수': cnt_list})

In [38]:
bill_df = pd.merge(df, bill_cnt, how='left', on='법안코드')

In [41]:
bill_df.to_csv('21th_bill_preprocessed.csv', encoding='utf-8-sig') #csv파일로 저장